In [1]:
import os
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import paddle

paddle.set_device("cpu")
paddle.set_default_dtype("float64")

In [2]:
def unpickle(file):
    import pickle

    with open(file, "rb") as fo:
        dict = pickle.load(fo, encoding="bytes")
    return dict


def getitem(dict, index):
    label = a[b"labels"][index]
    image_data = a[b"data"][index]
    red_channel = image_data[:1024].reshape((32, 32))
    green_channel = image_data[1024:2048].reshape((32, 32))
    blue_channel = image_data[2048:].reshape((32, 32))
    rgb_image = np.dstack((red_channel, green_channel, blue_channel))
    return rgb_image, label


label_names = unpickle("../data/cifar-10-batches-py/batches.meta")[b"label_names"]

In [3]:
class Reader(paddle.io.Dataset):
    def __init__(self, *path):
        super().__init__()
        data = unpickle(path[0])
        self.x = data[b"data"] / 255
        self.y = data[b"labels"]
        for i in range(1, len(path)):
            data = unpickle(path[i])
            self.x = np.concatenate((self.x, data[b"data"] / 255))
            self.y = np.concatenate((self.y, data[b"labels"]))

    def __getitem__(self, index):
        return self.x[index].reshape((3, 32, 32)), self.y[index]

    def __len__(self):
        return len(self.y)

In [4]:
class MyCNN(paddle.nn.Layer):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv0 = paddle.nn.Conv2D(3, 32, 3, 1, 0)
        self.pool0 = paddle.nn.MaxPool2D(2, 2)
        self.conv1 = paddle.nn.Conv2D(32, 128, 4, 1, 0)
        self.pool1 = paddle.nn.MaxPool2D(2, 2)
        self.conv2 = paddle.nn.Conv2D(128, 128, 5, 1, 0)
        self.pool2 = paddle.nn.MaxPool2D(2, 2)
        self.fc = paddle.nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv0(x)
        x = self.pool0(x)
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = x.flatten(1)
        # print(x.shape)
        x = self.fc(x)
        return x

In [5]:
model = MyCNN()
cross_entropy = paddle.nn.CrossEntropyLoss()
opt = paddle.optimizer.SGD(0.01, model.parameters())

model.train()
epoch = 30
batch = 10
train_loader = paddle.io.DataLoader(
    Reader(
        "../data/cifar-10-batches-py/data_batch_1",
        "../data/cifar-10-batches-py/data_batch_2",
        "../data/cifar-10-batches-py/data_batch_3",
        "../data/cifar-10-batches-py/data_batch_4",
        "../data/cifar-10-batches-py/data_batch_5",
    ),
    batch_size=batch,
    shuffle=True,
)

for e in range(epoch):
    full_loss = 0
    n_loss = 0
    n_eval = 0
    correct = 0
    for i, (x, y) in enumerate(train_loader):
        if i % 100 == 0:
            print("epoch", e, "\tbatch", i, end=" ")

        y_pred = model(x)
        loss = cross_entropy(y_pred, y)

        loss.backward()
        opt.step()

        full_loss += loss.numpy()
        n_loss += batch
        opt.clear_grad()

        mask = np.array(y_pred, copy=False).argmax(1) == y
        co = mask.sum().numpy()
        correct += co

        if i % 100 == 0:
            print(f"\tloss {loss.numpy():.2f} \taccuracy {co/batch:.2f}")
    print(f"Train Accuracy: {correct*100/n_loss}%")
    print(f"Epoch: {e}, Loss: {full_loss / n_loss}")

epoch 0 	batch 0 	loss 4.27 	accuracy 0.00


KeyboardInterrupt: 

In [ ]:
paddle.save(model.state_dict(), "train1.model")

In [7]:
m = MyCNN()
m.set_state_dict(paddle.load("train1.model"))
m.eval()
eval_loader = paddle.io.DataLoader(
    Reader("../data/cifar-10-batches-py/test_batch"), batch_size=1000, shuffle=True
)

accs = []
for i, (x, y) in enumerate(eval_loader):
    y_pred = m(x)
    mask = np.array(y_pred, copy=False).argmax(1) == y
    acc = mask.sum().numpy() / batch
    print(i, acc)
    accs.append(acc)
print(np.mean(accs))

0 93.1
1 94.0
2 93.9
3 93.0
4 94.7
5 93.1
6 93.2
7 93.4
8 93.0
9 92.4
93.38000000000001
